## Descriptors

#### A simple descriptor

In [ ]:
class Quantity:

    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:     
            return instance.__dict__[self.storage_name]
    

class House:
    rooms = Quantity('num_rooms')

house = House()
house.rooms = 3
print(house.rooms)
print(House.rooms)
         

In [ ]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
truffle = LineItem('White truffle', 100, 0)

## Automatic naming of storage attributes `__set_name__`
The `__get__` method is not needed anymore

In [ ]:
class Quantity:
    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'The value must be > 0'
            raise ValueError(msg)
        
class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, weight, price):
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price


## A new descriptor type

In [ ]:
import abc

class Validated(abc.ABC):
    def __set_name__(self, owner, name):
        self.storage_name = name
    
    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)
        instance.__dict__[self.storage_name] = value
    
    @abc.abstractmethod
    def validate(self, name, value):
        """return validated value or raise ValueError"""

In [ ]:
class Quantity(Validated):
    def validate(self, name, value):
        if value <= 0:
            msg = f'{name} value must be > 0'
            raise ValueError(msg)
        return value
    
class NonBlank(Validated):
    def validate(self, name, value: str):
        value = value.strip()
        if value == '':
            raise ValueError(f'{name} description must be non empty')
        return value


## Overriding vs non overriding descriptors

In [ ]:
# auxiliary classes not important

def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'
    
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print( f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

In [ ]:
class Overriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class OverridingNoGet:
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print(f'-> Managed.spam({display(self)})')

Overriding descriptor behaviour:

In [ ]:
obj = Managed()
obj.over
Managed.over
obj.over = 7
obj.over
obj.__dict__['over'] = 8
print(vars(obj))
obj.over

Overriding descriptor with no `__get__` behaviour:

In [ ]:
print(obj.over_no_get)
print(Managed.over_no_get)
obj.over_no_get = 7
print(obj.over_no_get)
obj.__dict__['over_no_get'] = 9
print(obj.over_no_get)
obj.over_no_get = 7


Non overriding descriptor

In [ ]:
obj = Managed()
obj.non_over
obj.non_over = 7
print(obj.non_over)
Managed.non_over
del obj.non_over
obj.non_over

In [ ]:
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

## Methods as descriptors
Methods are non overriding descriptors

In [ ]:
obj = Managed()
print(obj.spam)
obj.spam = 7
obj.spam

In [ ]:
import collections
class Text(collections.UserSTring):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    def reverse(self):
        return self[::-1]

## Descriptor usage tips

## Descriptor docstring and overriding deletion